<a href="https://colab.research.google.com/github/DarthCoder501/ML-AI-Projects/blob/main/Model_Market_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings as wr
wr.filterwarnings('ignore')

In [ ]:
# Load trained model
with open('xgb_model-FeatureSMOTE.pkl', 'rb') as f:
    model = pickle.load(f)

# Load CSV File
newDF = pd.read_csv('newData.csv')

# Load new data
X_new = newDF.iloc[:, 2:]

# Generate predictions
predictions = model.predict(X_new)

In [ ]:
# Define thresholds
crisis_threshold = 0.7

# Categorize periods
newDF['predicted_crisis'] = (predictions > crisis_threshold).astype(int)

# Define allocation strategy
def allocate_assets(row):
    if row['predicted_crisis'] == 1:  # Crisis period
        return {"cash": 70, "bonds": 20, "stocks": 10}
    else:  # Normal period
        return {"cash": 10, "bonds": 20, "stocks": 70}

newDF['allocation'] = newDF.apply(allocate_assets, axis=1)

In [ ]:
def simulate_strategy(data):
    # Starting portfolio value
    portfolio_value = 100000
    portfolio_history = []  # To track portfolio value over time
    prev_row = None  # To track previous row data

    for _, row in newDF.iterrows():
        allocation = row['allocation']  # Allocation dictionary
        market_data = {  # Extract relevant market data
            'stocks': row['MXUS_ma_4'],
            'bonds': row['LF94TRUU_ma_4'],
            'cash': 1.0  # Cash grows at a fixed rate
        }

        # Calculate returns based on previous market data
        returns = {
            'stocks': ((row['MXUS_ma_4'] / prev_row['MXUS_ma_4'] - 1) if prev_row is not None else 0),
            'bonds': ((row['LF94TRUU_ma_4'] / prev_row['LF94TRUU_ma_4'] - 1) if prev_row is not None else 0),
            'cash': 0.0006  # Weekly cash growth (adjust as needed)
        }

        # Adjust portfolio value based on allocation and returns
        portfolio_value += sum(
            (allocation[asset] / 100) * portfolio_value * returns.get(asset, 0)
            for asset in allocation
        )

        # Save portfolio value with date
        portfolio_history.append((row['Data'], portfolio_value))

        # Update previous row
        prev_row = row

    return portfolio_value, portfolio_history

In [ ]:
# Simulates Stragety
final_value, portfolio_history = simulate_strategy(data)
print("Final portfolio value:", final_value)

In [ ]:
# Visualize the portfolio's performance over time
dates, values = zip(*portfolio_history)  # Unpack dates and values
plt.plot(dates, values)
plt.title("Portfolio Value Over Time")
plt.xlabel("Date")
plt.ylabel("Portfolio Value")
plt.xticks(rotation=45)  # Rotate date labels for better readability
plt.show()